In [27]:
import math
import xml.etree.ElementTree as ET
import requests
import re
import pandas as pd
from openpyxl import load_workbook
from requests.adapters import HTTPAdapter
import urllib3
from urllib3 import Retry

def get_available_stations_id():
    """
    Retrives a list of all available stations and their id's - regardless of station type and size.
    :return: List of stations id's
    """
    stations = requests.get('https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/2.atom')
    root = ET.fromstring(stations.text)
    available_stations = []
    for child in root:
        if child.tag == '{http://www.w3.org/2005/Atom}entry':
            for subchild in child:
                if subchild.tag == '{http://www.w3.org/2005/Atom}id':
                    url_string = subchild.text
                    # extract station id from url which is not 1
                    station_id = re.search(r'(?<=station/)\d+', url_string).group(0)
                    available_stations.append(station_id)
    return available_stations


def get_data(stations, period, version='1.0', param='2'):
    """
    Retrives data from SMHI API, for a given station or list of stations and a given period.
    :param param: Parameter to retrieve data for, 2 = Daily average temperature
    :param version: API version
    :param stations: List of station id(s)
    :param period: Latest-hour or atest-day or latest-months or corrected-archive
    :return: List of lists, each list containing longitude coordinate, latitude coordinate, and a pandas dataframe with
    temperatures and dates.
    """
    result = []
    count = 0
    for station in stations:
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        temperatures = requests.get(
            'https://opendata-download-metobs.smhi.se/api/version/{}/parameter/{}/station/{}/period'
            '/{}/data.csv'.format(version, param, station, period))
        idx = temperatures.text.find('Från Datum Tid')
        temperatures_sliced = temperatures.text[idx:]
        longitude, latitude = float(temperatures.text.split(';')[17].strip().split('\n', 1)[0]), float(
            (temperatures.text.split(';')[18]).strip().split('\n', 1)[0])
        dataframe = pd.DataFrame([x.split(';') for x in temperatures_sliced.split('\n')[:-1]]).drop([0, 1, 4, 5, 6], axis=1)
        dataframe.columns = ['date', 'temperature']
        dataframe = dataframe.iloc[1: , :]
        result.append([longitude, latitude, dataframe])
        count += 1
    return result

In [28]:
stations_ids = get_available_stations_id()
# data = get_data([stations[0]], 'corrected-archive')
stations_data = get_data([stations_ids[11], stations_ids[600]], 'corrected-archive')

In [29]:
def qut(temp):
    # Constants: [W/(m^2 K)]
    u_wall = 0.13
    u_window = 4  # Rimligt, värde
    u_roof = 0.13
    u_floor = 0.12
    u_door = 1
    short_wall = 5  # m
    long_wall = 15  # m
    height = 6  # m 3 kankse?
    area_floor = short_wall * long_wall  # m^2
    area_walls = (2 * height * short_wall) + (2 * height * long_wall)  # m^2
    antal_fönster = 10
    antal_dörrar = 2
    area_windows = 1.5 * 1 * antal_fönster  # m^2
    alpha = math.pi / 6  # Rad
    area_roof = (short_wall * long_wall) / (math.cos(alpha))  # m^2
    area_doors = 2.1 * 1 * antal_dörrar  # m^2
    t_inne = 20  # C
    enhetsomvanlding = (3600 * 24 * 10**(-6)) / 3.6  # kWh/dag
    q_wall = u_wall * area_walls * (t_inne - temp)
    q_roof = u_roof * area_roof * (t_inne - temp)
    q_floor = u_floor * area_floor * (t_inne - temp)
    q_window = u_window * area_windows * (t_inne - temp)
    q_door = u_door * area_doors * (t_inne - temp)
    return (q_wall + q_roof + q_floor + q_window + q_door) * enhetsomvanlding  # kWh


def dates():
    result = []
    for i in range(2013, 2023):
        for j in range(1, 13):
            if len(str(j)) == 1:
                j = '0' + str(j)
            result.append(str(i) + '-' + str(j))
    result.remove('2013-01')
    result.remove('2013-02')
    result.remove('2013-03')
    result.pop()
    result.pop()
    return result

def vinst(Q, pris, t_ut, t=20, x = 1):
    t = t + 273.15
    t_ut = t_ut + 273.15
    scop = (t / (t - t_ut)) * x
    if x > 1:
        scop = 4.3
    return Q * (pris/100) * (1 - (1/scop)) # 0 < vinst < Q*pris

# def vinst1(Q, pris, scop = 4.3):
#    return Q * (pris/100) * (1 - (1/scop))

In [30]:
for long, lat, df in stations_data:
    #print(long, lat)
    #display(df[19590:19730])

SyntaxError: incomplete input (806444149.py, line 3)

In [31]:
datum = dates()
pris_omrade1 = {}
pris_omrade2 = {}
pris_omrade3 = {}
pris_omrade4 = {}
path = '/Users/dennisnilsson/IdeaProjects/michel/manadsvarden_2022m10_.xlsx'
wb = load_workbook(path)
sheet = wb['Anvisat avtal']
all_rows = list(sheet.rows)
omraden = []
for row in all_rows[1:44]:
    if row[0].value == 'Större hushåll':
        res = []
        for cell in row[3:(121 - 3)]:
            res.append(cell.value)
        omraden.append(res)
for i in range(0, len(datum)):
    pris_omrade1[datum[i]] = omraden[0][i]
    pris_omrade2[datum[i]] = omraden[1][i]
    pris_omrade3[datum[i]] = omraden[2][i]
    pris_omrade4[datum[i]] = omraden[3][i]
q_ut1 = []

for temp in stations_data[0][2].iloc[:, 1]:
    if temp is not None and float(temp) < 20:
        q_ut1.append(qut(float(temp)))
    else:
        q_ut1.append(0)
stations_data[0][2]['Q'] =  q_ut1
# print(sum(stations_data[0][2].iloc[19773-365:, 3]))
vinst_list= []
extracted_dates = []
df = stations_data[0][2]
is_started = False
is_finished = False
for index, row in df.iterrows():
    if row['Q'] == 0:
        continue
    elif is_started is False and is_finished is False:
        if datum[0] in row['date']:
            is_started = True
    elif is_started is True and is_finished is False:
        vinst_list.append(vinst(row['Q'], pris_omrade3[row['date'][:7]], float(row['temperature']), x = 2))
        extracted_dates.append(row['date'])
        if (datum[-1][:-1] + '1') in row['date']:
            is_finished = True
    else: break
#print(stations_data[0][2])




In [32]:
import plotly.express as px

s = pd.Series(vinst_list).cumsum()
add = {'Datum': extracted_dates, 'Kumulativ vinst': s-105889}
df2 = pd.DataFrame(add)
fig = px.line(df2, x='Datum', y='Kumulativ vinst')
fig.show()
fig.write_image("images/en_matpunk.png")



In [33]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

datum = dates()

def populate_prices():
    pris_omrade1 = {}
    pris_omrade2 = {}
    pris_omrade3 = {}
    pris_omrade4 = {}
    path = '/Users/dennisnilsson/IdeaProjects/michel/manadsvarden_2022m10_.xlsx'
    wb = load_workbook(path)
    sheet = wb['Anvisat avtal']
    all_rows = list(sheet.rows)
    omraden = []
    """path2 = '/Users/dennisnilsson/IdeaProjects/michel/arsvarde_2.xlsx'
    wb2 = load_workbook(path2)
    sheet2 = wb2['Tillsvidarepris']
    all_rows2 = list(sheet2.rows)
    for rad in all_rows2[1:124]:
        if rad[0].value == 'Villa med elvärme':
            res = []
            for box in rad[3:16]:
                if box.value is not None:
                    res.append(box.value)
            omraden.append(res)"""
    for rad in all_rows[1:44]:
        if rad[0].value == 'Villa med elvärme':
            res = []
            for box in rad[3:(121 - 3)]:
                if box.value is not None:
                    res.append(box.value)
            omraden.append(res)
    for i in range(0, len(datum)):
        pris_omrade1[datum[i]] = omraden[0][i]
        pris_omrade2[datum[i]] = omraden[1][i]
        pris_omrade3[datum[i]] = omraden[2][i]
        pris_omrade4[datum[i]] = omraden[3][i]
    return [pris_omrade1, pris_omrade2, pris_omrade3, pris_omrade4]

def get_price_area(long, omraden):
    """
    Retunerar en av de givna listorna.
    :param long: Longitud koordinat
    :param omraden: En lista av dict med områdena.
    :return: Korrekt dict.
    """
    if long > 64.42:
        return omraden[0]
    elif long > 61.24:
        return omraden[1]
    elif long > 57.15:
        return omraden[2]
    else:
        return omraden[3]

def get_qut(df):
    q_ut1 = []
    for temp in df.iloc[:, 1]:
        if temp is not None and temp != '' and temp != ' ':
            if float(temp) < 20:
                q_ut1.append(qut(float(temp)))
            else: q_ut1.append(0) # Should correct array size, to match df when creating new col.
        else:
            q_ut1.append(0)
    return q_ut1

def get_vinst(dataframe1, pris_omrade):
    vinst_list1= []
    extracted_dates1 = []
    is_started1 = False
    is_finished1 = False
    for w, rad in dataframe1.iterrows():
        if rad['Q'] == 0:
            continue
        elif is_started1 is False and is_finished1 is False:
            if datum[0] in rad['date']:
                is_started1 = True
        elif is_started1 is True and is_finished1 is False:
            vinst_list1.append(vinst(rad['Q'], pris_omrade[rad['date'][:7]], float(rad['temperature']),  x = 2))
            extracted_dates1.append(rad['date'])
            if (datum[-1][:-1] + '1') in rad['date']:
                is_finished1 = True
        else: break
    return vinst_list1

def intersect_list(list1, list2):
    return [n for n in list1 if
            any(m in n for m in list2)]

def retrieve():
    stations = get_available_stations_id()
    #whitelist = ['154860', '188800', '188790', '158990', '85600', '97280', '92480', '117330', '178740', '75040', '99090', '134590', '53640', '53390', '178790', '82000', '73250', '156990', '74600', '74510', '144530', '95230', '167710', '97390', '53770', '159880', '92400', '92410', '97630', '84660', '94370', '106070', '84340', '105550', '97260', '82540', '97190', '125470', '74080', '98040', '98610', '96610', '136020', '106080', '116650', '151290', '151280', '85160', '53650', '142030', '163680', '107040', '178820', '83640', '155710', '75140', '92170', '92130', '132110', '78050', '161940', '170630', '155900', '72400', '54300', '83170', '105310', '105280', '105260', '72450', '138270', '91130', '96640', '106040', '92370', '127130', '161930', '145450', '82640', '157870', '78240', '133030', '150210', '84620', '62600', '136610', '123460', '94390', '115630', '160600', '73430', '115040', '156850', '107120', '94140', '116480', '116490', '156770', '156790', '81320', '91370', '105640', '75120', '123060', '96120', '132240', '116240', '103050', '132590', '115230', '115220', '107440', '53710', '64330', '66470', '96140', '98630', '97370', '82380', '96230', '96190', '181970', '63470', '98640', '72120', '107330', '86090', '62520', '83090', '52230', '52240', '105370', '98620', '162800', '95320', '94430', '107140', '107130', '125020', '118610', '85410', '122360', '136491', '74420', '104390', '96040', '112510', '106100', '23059', '137080', '93580', '148050', '148040', '104090', '126630', '126300', '63530', '134130', '151550', '73390', '84390', '160750', '160740', '79580', '78550', '134410', '134460', '94410', '82030', '82110', '63340', '155790', '73170', '135640', '76420', '62550', '159770', '85450', '89240', '89230', '127600', '103290', '72070', '112170', '108300', '105030', '84610', '82200', '105360', '95380', '99150', '122610', '145130', '84690', '147570', '147560', '76430', '87360', '98180', '103090', '103100', '172740', '106170', '144300', '144310', '180760', '180740', '125501', '54650', '106670', '106680', '106630', '107400', '107420', '84520', '74350', '72630', '71420', '72420', '74180', '62260', '135460', '135440', '62400', '62390', '62410', '62420', '54310', '114410', '62530', '64020', '163950', '163960', '170670', '161780', '108600', '180710', '87150', '87140', '62080', '123250', '74410', '123340', '161650', '62030', '62040', '155940', '155970', '155960', '138400', '138390', '78140', '85210', '78250', '73230', '52560', '68550', '68560', '86200', '140360', '140460', '75520', '146070', '146050', '117440', '75410', '125450', '125440', '98290', '74480', '88590', '89560', '149700', '149710', '149160', '83190', '128370', '127380', '85180', '75380', '63080', '63160', '6057', '81630', '91520', '128560', '145340', '133050', '95020', '83610', '91660', '162970', '53630', '117370', '102540', '53530', '105230', '112540', '147270', '115460', '169880']
    #stations_filtered = intersect_list(stations, whitelist)
    #print(stations_filtered)
    #print(len(stations_filtered)) # Ska vara 301
    stations_joined = get_data(stations, 'corrected-archive')
    return stations_joined, stations

def get_breakpoints(amount):
    stations_joined, stations = retrieve()
    omrade = populate_prices()
    returned_data = [[], [], []]
    throwCount = 0
    #used_stations = []
    #thrown_stations = []
    count = 0
    #dates_used = []
    for station_info in stations_joined:
        latitude = station_info[0]
        longitude = station_info[1]
        df_station = station_info[2]
        strang = df_station.iloc[-1, 0][:4]
        if strang.isnumeric():
            if int(datum[0][:4]) > int(strang):
                #print('throwing away')
                # print(df_station.iloc[0, 0] + " to " + df_station.iloc[-1, 0])
                throwCount+= 1
                #print(throwCount)
                #thrown_stations.append(stations[count])
                continue
        df_station['Q'] = get_qut(df_station)
        vinst_av_df = get_vinst(df_station, get_price_area(longitude, omrade))
        #if len(vinst_av_df) == 0:
        #     continue
        d = pd.Series(vinst_av_df).cumsum()
        bound = len(d.where(d <= amount).dropna())
        returned_data[0].append(latitude)
        returned_data[1].append(longitude)
        returned_data[2].append(bound)
        #dates_used.append(df_station.iloc[0, 0] + " to " + df_station.iloc[-1, 0])
        #used_stations.append(stations[count])
        count += 1
    return returned_data#, used_stations, dates_used, thrown_stations


In [34]:
import pickle

data = get_breakpoints(105889/2)
# print(data)
# print(used)

#with open("data", "wb") as fp:   #Pickling
#    pickle.dump(data, fp)
"""""
with open("used", "wb") as fp:   #Pickling
    pickle.dump(used, fp)
with open("dates", "wb") as fp:   #Pickling
    pickle.dump(dates, fp)
with open("thrown", "wb") as fp:   #Pickling
    pickle.dump(thrown, fp)"""

/var/folders/2q/b21j6t1s6sq886fhsmgdcq9r0000gn/T/ipykernel_2508/2270130766.py:126: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/var/folders/2q/b21j6t1s6sq886fhsmgdcq9r0000gn/T/ipykernel_2508/2270130766.py:126: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/var/folders/2q/b21j6t1s6sq886fhsmgdcq9r0000gn/T/ipykernel_2508/2270130766.py:126: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/var/folders/2q/b21j6t1s6sq886fhsmgdcq9r0000gn/T/ipykernel_2508/2270130766.py:126: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

/var/folders/2q/

'""\nwith open("used", "wb") as fp:   #Pickling\n    pickle.dump(used, fp)\nwith open("dates", "wb") as fp:   #Pickling\n    pickle.dump(dates, fp)\nwith open("thrown", "wb") as fp:   #Pickling\n    pickle.dump(thrown, fp)'

In [35]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

""""with open("data", "rb") as fp:   # Unpickling
    saved_data = pickle.load(fp)
print(data)
with open("used", "rb") as fp:   # Unpickling
    saved_used = pickle.load(fp)
with open("dates", "rb") as fp:   # Unpickling
    dates = pickle.load(fp)
with open("thrown", "rb") as fp:   # Unpickling
    thrown = pickle.load(fp)
"""""

pio.templates.default = "simple_white"
data_transformed = pd.DataFrame({'Latitud': data[0], 'Longitud': data[1], 'År': [2*(dagar/365) for dagar in data[2]]})
fig = px.density_mapbox(data_transformed, lat='Latitud', lon='Longitud', z='År', mapbox_style="open-street-map", opacity=0.2, zoom= 5, height=1600, width=900, radius=50, center=dict(lat=62.7, lon=17.5))
fig.add_trace(
    go.Scattermapbox(
        lat=data_transformed["Latitud"],
        lon=data_transformed["Longitud"],
        mode="markers",
        showlegend=False,
        hoverinfo="skip",
        marker={
            "color": data_transformed["År"],
            "size": data_transformed["År"].fillna(0),
            "coloraxis": "coloraxis",
            # desired max size is 15. see https://plotly.com/python/bubble-maps/#united-states-bubble-map
            "sizeref": (data_transformed["År"].max()) / 12 ** 2,
            "sizemode": "area",
        },
    )
)

#fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
 #                  'paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()
fig.write_image("images/BV_4.3_52945_noFilter_TOTALTID.png")



In [44]:
mCOP = 4.76
T_h = (273 + 35)
T_l = (273 + 7)
cCOP = T_l / (T_h - T_l)
x = mCOP / cCOP
print(cCOP, x)

10.0 0.476


In [75]:
print(105889/2)

52944.5
